In [1]:
!pip install msal msal_extensions
!pip install datasets

In [2]:
# onedrive 관련
from io import StringIO
from msal import PublicClientApplication
import requests
import os
from msal_extensions import FilePersistence, PersistedTokenCache
import pandas as pd

# 모델 관련
import torch
from google.colab import drive
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
from datasets import load_dataset, Dataset
from tqdm.auto import tqdm

In [3]:
def get_ms_token(client_id, authority, scopes):
    # 캐시 파일 경로 설정
    cache_file_path = os.path.expanduser('~/.msal_cache.json')

    # MSAL Extensions를 사용하여 파일 기반 캐시 생성
    persistence = FilePersistence(cache_file_path)
    token_cache = PersistedTokenCache(persistence)

    # MSAL 앱 생성
    app = PublicClientApplication(client_id, authority=authority, token_cache=token_cache)

    # 캐시에서 기존 계정 확인
    accounts = app.get_accounts()
    if accounts:
        # 첫 번째 계정 선택 (여러 계정이 있을 경우 적절히 선택)
        result = app.acquire_token_silent(scopes, account=accounts[0])
        if 'access_token' in result:
            print('캐시된 토큰을 사용합니다.')
        else:
            print('캐시에서 유효한 토큰을 찾을 수 없습니다. 인증을 진행합니다...')
    else:
        # 디바이스 코드 플로우를 통한 새 인증 진행
        flow = app.initiate_device_flow(scopes=scopes)
        if 'user_code' not in flow:
            raise ValueError('디바이스 플로우 생성에 실패했습니다. 설정을 확인하세요.')
        print(f"다음 URL로 이동하여 코드를 입력하세요: {flow['verification_uri']}")
        print(f"인증 코드: {flow['user_code']}")
        result = app.acquire_token_by_device_flow(flow)

    if 'access_token' in result:
        print('인증에 성공했습니다!')
        headers = {'Authorization': f"Bearer {result['access_token']}"}
        return headers
    else:
        print('인증에 실패했습니다.')
        return None

In [4]:
def list_onedrive_files(headers, target_name, folder_id=None):
    """
    OneDrive 폴더의 파일 목록을 가져옵니다.

    Parameters:
        headers (str): MSAL을 통해 얻은 인증 토큰
        target_name(str): 내가 찾고자 하는 파일의 이름
        folder_id (str): 폴더의 ID (None일 경우 루트 폴더)

    Returns:
        dict: 파일 이름과 파일 ID의 매핑
    """
    base_url = "https://graph.microsoft.com/v1.0/me/drive"
    url = f"{base_url}/items/{folder_id}/children" if folder_id else f"{base_url}/root/children"

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        folders = response.json().get('value', [])
        # 확장자가 없는 폴더만 사전에 추가
        folder_mapping = {folder['name']: folder['id'] for folder in folders}
        print(folder_mapping)
        for name in folder_mapping:
            if name == target_name:
                return folder_mapping[name]
            elif name.find('.') == -1:
                result = list_onedrive_files(headers, target_name, folder_mapping[name])
                if result:
                    return result

    elif response.status_code == 404:
        print(f"폴더를 찾을 수 없습니다: {folder_id}")
        return False

    else:
        print(f"폴더 파일 목록을 가져오는 데 실패했습니다: {response.status_code} - {response.text}")
        return False

In [5]:
def load_csv_from_onedrive(headers, file_id) -> pd.DataFrame:
    url = f"https://graph.microsoft.com/v1.0/me/drive/items/{file_id}/content"
    response = requests.get(url, headers=headers, stream=True)

    if response.status_code == 200:
        csv = pd.read_csv(StringIO(response.text))
        print(f"파일 탐색 성공")
        return csv
    else:
        print(f"파일 다운로드 실패: {response.status_code} - {response.text}")

In [6]:
# Azure 앱 정보
GRAPH_API_URL = 'https://graph.microsoft.com/v1.0'
CLIENT_ID = 'ef053b61-d7f1-4942-97d4-bb79fa475a01'  # 앱 등록에서 가져온 클라이언트 ID
AUTHORITY = 'https://login.microsoftonline.com/f09a4ef3-978d-434e-89da-a29b9f9f3c32'  # 테넌트 ID 또는 'common'
SCOPES = ['Files.ReadWrite.All']  # 필요 권한 설정

In [7]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
headers = get_ms_token(CLIENT_ID, AUTHORITY, SCOPES)

file_id = list_onedrive_files(headers, 'test.csv')

캐시된 토큰을 사용합니다.
인증에 성공했습니다!
{'.idea': '01UUMNEVO7JTDLP2VNH5H2DQBVLVWTQFMC', '첨부 파일': '01UUMNEVPWL73GRXSYZNDZUQO7WAMFJK5U', 'AI_Bio_Research': '01UUMNEVLFHOTOGTSKKBEI7SGXFGRI3OYV', 'dacon': '01UUMNEVP3V7HTIIDC4ND2OGQA2MDF5DEB'}
{}
{'1000_이론 및 실습': '01UUMNEVIOEEJ5BSHJDRDID2TYJ5XBI5FL', '2000_논문 리뷰': '01UUMNEVM6QJ2EX34MGJCK3CW4P3ZLTLRY', '3000_연구 수행': '01UUMNEVKW4UAVQVOYXZFLLX7OBYGDEPD5', '5000_코드 및 데이터 관리': '01UUMNEVPPHAEXKXCXLRC2BJGY34Y4LP3Y', '6000_블로그 관리': '01UUMNEVJOXNVK6OM3DFB3IHVUQCCRV5OS', '7000_컨퍼런스 및 네트워킹': '01UUMNEVIGWDP3RQ2VKNDIU56S6IRD7GMY', '8000_참고 자료': '01UUMNEVMVOBJCEVTAFRFIFBKXT4H5BAE4', '신약_AI_개발자로_거듭나기_5개년_계획.xlsx': '01UUMNEVOOB3TVWMZNNZBLLSBKTADL3TQ7'}
{}
{'2100_신약 개발 AI': '01UUMNEVISQIWEGPWPN5BIDUDQCORZHVMT', '2200_단백질 - 화합물 결합 예측': '01UUMNEVPLSHJGBXWHJBC2QHUXCWFMBHRA', '2300 MD Simulation': '01UUMNEVO7ASKOUGXEWBFZLUDMCUOMBK6H', '2900_기타 논문': '01UUMNEVLZS4GQQK6KEZDLVCJLMCOYOEP3'}
{'논문_A.md': '01UUMNEVOXMVZUKCUFMNG3NLIQV7UHNGI3', '논문_B.md': '01UUMNEVPYXDZ53YJJ6RH3VUMUO

In [9]:
# 모델이 저장된 경로 확인
model_name = '/content/drive/MyDrive/Colab/korean_review_decode_baseline_model'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [10]:
# GPU 사용 가능하면 GPU로 진행
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [11]:
# 데이터셋 로드
df = load_csv_from_onedrive(headers, file_id)
df.head()

파일 탐색 성공


,ID,input
0,TEST_0000,녀뮨넒뭅 만죡숭러윤 효템뤼에오. 푸싸눼 옰면 콕 츄쩐학꼬 싶은 콧쉰웨오. 췌꾜윕뉘댜...
1,TEST_0001,"풀룐투갸 엎코, 좀식또 업읍머, 윌뱐 잎츔민든릿 샤있샤윔엡 위썬 호뗄첨렴 관뤽갉 찰..."
2,TEST_0002,쥔차 붉찐졀행욘. 삶먼섶 멂묽럿턴 혹텔 중웨 쬐약위였습뉜따. 칙어뉜쥐 샤쨩윈쥐 쩨끄...
3,TEST_0003,붊 맛짚~~ 글련뎨 방움잃 뮈흙퍄녜용. 충칸 쏘움광 팔쿄닛갸 잊중짱임 야뉘럇셧 팜몌...
4,TEST_0004,빻 샹택는 쥔쨔 폐헐 칙젓뉜테 쩐맣은 죠하욧. 뽀읾럭카 알쥬 찬쟌합꿰 똘앝썬 츄어서...


In [12]:
# 난독화된 문장을 정상적인 문장으로 변환하는 함수
def decode_text(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True, padding="max_length")
    inputs = inputs.to(device)
    outputs = model.generate(inputs["input_ids"], max_length=128, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [13]:
from tqdm.auto import tqdm
tqdm.pandas()

# decode_text 적용 (기존 코드와 동일)
df['input'] = df['input'].progress_apply(decode_text)

# 컬럼 이름 변경 (기존 코드와 동일)
df.rename(columns={'input': 'output'}, inplace=True)

  0%|          | 0/1689 [00:00<?, ?it/s]

In [16]:
# 결과를 새로운 csv 파일로 저장
output_csv_path = '/content/drive/MyDrive/Colab/result.csv'
df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
print(f"결과가 {output_csv_path}에 저장되었습니다.")

결과가 /content/drive/MyDrive/Colab/result.csv에 저장되었습니다.
